In [1]:
from sympy import *
import numpy as np

x = Symbol('x')
y = Symbol('y')
f = 100*(y-x**2)**2+(1-x)**2

In [2]:
def hessian_manual(f, varlist):
    n = len(varlist)
    hessian = zeros(n)
    for i in range(n):
        for j in range(i, n):
            hessian[i, j] = f.diff(varlist[i]).diff(varlist[j])
    return hessian

hessian_manual(f, [x, y])

Matrix([
[1200*x**2 - 400*y + 2, -400*x],
[                    0,    200]])

In [3]:
def gradient_descent(f, start, learning_rate, iterations):
    grad_x = diff(f, x)
    grad_y = diff(f, y)
    
    grad_x_func = lambdify((x, y), grad_x)
    grad_y_func = lambdify((x, y), grad_y)
    
    current_x, current_y = start
    trajectory = [(current_x, current_y)]
    for _ in range(iterations):
        grad_x_val = grad_x_func(current_x, current_y)
        grad_y_val = grad_y_func(current_x, current_y)
        
        current_x -= learning_rate * grad_x_val
        current_y -= learning_rate * grad_y_val
        
        trajectory.append((current_x, current_y))

    return current_x, current_y, trajectory

min_x, min_y, trajectory = gradient_descent(f, (0.5, 0.5), 0.005, 100)
min_x, min_y

(0.40451478920271566, 0.19989881420134903)

In [4]:
def newton(f, start, learning_rate, iterations):
    gradient = Matrix([diff(f, val) for val in [x, y]])
    hessian = hessian_manual(f, [x, y])

    grad_func = lambdify([x, y], gradient, "numpy")
    hessian_func = lambdify([x, y], hessian, "numpy")

    current_x, current_y = start
    trajectory = [(current_x, current_y)]

    for i in range(iterations):
        grad_val = np.array(grad_func(current_x, current_y), dtype=float).flatten()
        hessian_val = np.array(hessian_func(current_x, current_y), dtype=float)

        try:
            hessian_inv = np.linalg.inv(hessian_val)
            step = hessian_inv @ grad_val
        except np.linalg.LinAlgError:
            print(f"Abbruch bei Iteration {i}: Hesse-Matrix ist nicht invertierbar.")
            break

        current_x -= learning_rate * step[0]
        current_y -= learning_rate * step[1]

        trajectory.append((current_x, current_y))

    return current_x, current_y, trajectory

min_x, min_y, trajectory = newton(f, (0.5, 0.5), 1, 100)
min_x, min_y, trajectory

(np.float64(0.7335153249168256),
 np.float64(0.536226725563526),
 [(0.5, 0.5),
  (np.float64(0.5098039215686274), np.float64(0.25)),
  (np.float64(0.5143877627196569), np.float64(0.2599000384467512)),
  (np.float64(0.5188934818669486), np.float64(0.26459477043573404)),
  (np.float64(0.5232818860917139), np.float64(0.26925044552400534)),
  (np.float64(0.5275595229114539), np.float64(0.27382393231170143)),
  (np.float64(0.531732043281134), np.float64(0.2783190502145609)),
  (np.float64(0.5358046620853659), np.float64(0.28273896585192976)),
  (np.float64(0.5397822000329746), np.float64(0.2870866359124132)),
  (np.float64(0.543669123255468), np.float64(0.2913648234724382)),
  (np.float64(0.5474695779721184), np.float64(0.29557611558136926)),
  (np.float64(0.5511874209680012), np.float64(0.2997229388049694)),
  (np.float64(0.5548262464822347), np.float64(0.3038075730333566)),
  (np.float64(0.55838941000619), np.float64(0.30783216378556544)),
  (np.float64(0.561880049411657), np.float64(0.31